In [1]:
import os
import psycopg2
import pandas as pd
import json

In [2]:
def connect_db():
    '''
    Creating connection to DB
    '''
    try:
        conn = psycopg2.connect(
                host=os.environ['MID_PROJ_DB_HOST'],
                database=os.environ['MID_PROJ_DB'],
                user=os.environ['MID_PROJ_DB_USERNAME'],
                password=os.environ['MID_PROJ_DB_PASSWORD'],
                port='5432')
        print('Db Connected succsefuly')
        return conn
    except (Exception, Error) as error:
        print("Error while connecting")

In [3]:
def query_db(query,conn):
    cursor = conn.cursor()
    cursor.execute(query)
    return {'data': cursor.fetchall(), 'columns': [desc[0].strip() for desc in cursor.description]} 

In [4]:
def write_to_file(csv_name,query_output):
    df = pd.DataFrame(query_output['data'])
    df.columns = query_output['columns']
    df.to_csv(csv_name + '.csv', sep=",", index = False)

In [5]:
conn = connect_db()

Db Connected succsefuly


In [19]:
flights_jan_dec_19_query="""select * from flights
where fl_date > '2018-12-31' 
and fl_date < '2019-02-01' 
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10)
union 
select * from flights 
where  fl_date > '2019-11-30' 
and fl_date < '2020-01-01'
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10);
"""
flights_jan_dec_19 = query_db(flights_jan_dec_19_query,conn)

In [20]:
flights_jan_dec_18_query="""select * from flights
where fl_date > '2017-12-31' 
and fl_date < '2018-02-01' 
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10)
union 
select * from flights 
where  fl_date > '2018-11-30' 
and fl_date < '2019-01-01'
and origin_city_name in (select origin_city_name
from flights
group by origin_city_name
order by COUNT(origin_city_name) desc 
limit 10)
and dest_city_name in (select dest_city_name
from flights
group by dest_city_name
order by COUNT(dest_city_name) desc 
limit 10);
"""
flights_jan_dec_18 = query_db(flights_jan_dec_18_query,conn)

In [21]:
write_to_file('flights_jan_dec_19',flights_jan_dec_19)

In [2]:
write_to_file('flights_jan_dec_18',flights_jan_dec_18)